### **Packages:**


---




In [0]:
# Importing Data Analysis Librarys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### **Data setup**:


---

In [18]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [3]:
!pip install kaggle

In [12]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
# token = {'username':'hemantkshirsagar8','key':''}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [24]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [26]:
!kaggle competitions download -c mlsp-2014-mri -p /content/data

  0% 0.00/2.04k [00:00<?, ?B/s]
100% 2.04k/2.04k [00:00<00:00, 3.92MB/s]
 70% 10.0M/14.3M [00:00<00:00, 48.6MB/s]
100% 14.3M/14.3M [00:00<00:00, 56.5MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 76.7MB/s]
  0% 0.00/121k [00:00<?, ?B/s]
100% 121k/121k [00:00<00:00, 128MB/s]
 94% 180M/192M [00:02<00:00, 92.4MB/s]
100% 192M/192M [00:02<00:00, 90.3MB/s]


In [27]:
!unzip /content/data/\*.zip -d /content/data/

Archive:  /content/data/HelperCode.zip
  inflating: /content/data/load_features.R  
  inflating: /content/data/load_features.m  

Archive:  /content/data/Train.zip
  inflating: /content/data/train_FNC.csv  
  inflating: /content/data/train_labels.csv  
  inflating: /content/data/train_SBM.csv  

Archive:  /content/data/AdditionalInformation.zip
  inflating: /content/data/load_maps.R  
  inflating: /content/data/gm_sMRI_ica_maps.pdf  
  inflating: /content/data/rs_fMRI_ica_maps.pdf  
  inflating: /content/data/aal_labels_naming.m  
  inflating: /content/data/load_AAL.m  
  inflating: /content/data/load_maps.m  
  inflating: /content/data/show_maps.m  
  inflating: /content/data/comp_ind_fMRI.csv  
  inflating: /content/data/comp_ind_sMRI.csv  
  inflating: /content/data/rs_fMRI_FNC_mapping.csv  
  inflating: /content/data/aal_labels.nii  
  inflating: /content/data/gm_sMRI_ica_maps.nii  
  inflating: /content/data/rs_fMRI_ica_maps.nii  

Archive:  /content/data/Test.zip
  inflating: /co

In [0]:
!rm -r /content/data/*.zip
# !rm -r /content/data/*.m
# !rm -r /content/data/*.R
# !rm -r /content/data/*.nii

### **About Data:**



---



**Train.zip:**

**train_labels.csv** - Labels for the training set. The labels are indicated in the "Class" column. 0 = 'Healthy Control', 1 = 'Schizophrenic Patient'

**train_FNC.csv** - FNC features for the training set. These are correlation values. They describe the connection level between pairs of brain maps over time.

**train_SBM.csv** - SBM features for the training set. These are standardized weights. They describe the expression level of ICA brain maps derived from gray-matter concentration.

**Test.zip:**

**test_FNC.csv** - FNC features for the test set. Test subject labels have been removed. Your task is to predict these unknown labels from the provided features.

**test_SBM.csv** - SBM features for the test set. Test subject labels have been removed.


**About FNC Features:**

Functional Network Connectivity (FNC) are correlation values that summarize the overall connection between independent brain maps over time. Therefore, the FNC feature gives a picture of the connectivity pattern over time between independent networks (or brain maps). The provided FNC information was obtained from functional magnetic resonance imaging (fMRI) from a set of schizophrenic patients and healthy controls at rest, using group independent component analysis (GICA). The GICA decomposition of the fMRI data resulted in a set of brain maps, and corresponding timecourses. 

These timecourses indicated the activity level of the corresponding brain map at each point in time. The FNC feature are the correlations between these timecourses. In a way, FNC indicates a subject's overall level of 'synchronicity' between brain areas. Because this information is derived from functional MRI scans, FNCs are considered a functional modality feature (i.e., they describe patterns of the brain function). More about FNCs can be found here: [FNC paper](http://cercor.oxfordjournals.org/content/early/2012/11/09/cercor.bhs352.abstract).

**About SBM Loadings:**

Source-Based Morphometry (SBM) loadings correspond to the weights of brain maps obtained from the application of independent component analysis (ICA) on the gray-matter concentration maps of all subjects. Gray-matter corresponds to the outer-sheet of the brain; it is the brain region in which much of the brain signal processing actually occurs. In a way, the concentration of gray-matter is indicative of the "computational power" available in a certain region of the brain. Processing gray-matter concentration maps with ICA yields independent brain maps whose expression levels (i.e., loadings) vary across subjects. 

Simply put, a near-zero loading for a given ICA-derived brain map indicates that the brain regions outlined in that map are lowly present in the subject (i.e., the gray-matter concentration in those regions are very low in that subject). Because this information is derived from structural MRI scans, SBM loadings are considered a structural modality feature (i.e., they describe patterns of the brain structure). More about SBM loadings can be found here: [SBM paper](http://www.ncbi.nlm.nih.gov/pubmed/22470337).

Further details about data can be found here, at [kaggle competition](https://www.kaggle.com/c/mlsp-2014-mri/data).


Done.

### **Data Analysis:**


---



In [0]:
train_FNC = pd.read_csv('./data/train_FNC.csv')
train_SBM = pd.read_csv('./data/train_SBM.csv')
y = pd.read_csv('./data/train_labels.csv')

In [30]:
train_FNC.head()

,Id,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,FNC10,FNC11,FNC12,FNC13,FNC14,FNC15,FNC16,FNC17,FNC18,FNC19,FNC20,FNC21,FNC22,FNC23,FNC24,FNC25,FNC26,FNC27,FNC28,FNC29,FNC30,FNC31,FNC32,FNC33,FNC34,FNC35,FNC36,FNC37,FNC38,FNC39,...,FNC339,FNC340,FNC341,FNC342,FNC343,FNC344,FNC345,FNC346,FNC347,FNC348,FNC349,FNC350,FNC351,FNC352,FNC353,FNC354,FNC355,FNC356,FNC357,FNC358,FNC359,FNC360,FNC361,FNC362,FNC363,FNC364,FNC365,FNC366,FNC367,FNC368,FNC369,FNC370,FNC371,FNC372,FNC373,FNC374,FNC375,FNC376,FNC377,FNC378
0,120873,0.34312,0.045761,-0.13112,0.15034,0.18082,0.289160,0.069545,-0.052489,0.124000,0.477620,0.165250,0.152740,0.270490,-0.023773,0.057374,-0.13873,0.24606,-0.036653,0.055729,0.195540,0.11568,0.353010,0.33846,0.22753,-0.065594,0.086119,-0.116460,0.41478,-0.35864,0.036615,0.35761,0.57461,0.21516,-0.156410,0.069346,0.39911,-0.086613,0.054857,0.192330,...,-0.29504,0.10551,0.30505,0.432070,0.556880,0.46394,0.018254,-0.188530,0.086159,-0.238100,0.25122,0.58942,0.542530,0.183010,-0.23049,0.093536,-0.024040,0.470320,0.29130,0.098155,0.047888,-0.134690,-0.23674,0.348840,0.40354,0.28671,0.27811,0.31830,0.37908,-0.060204,0.18743,0.163770,0.17686,0.074728,0.148600,0.343230,0.17565,0.52035,0.030613,0.30774
1,135376,0.28790,0.102570,-0.32343,-0.22776,0.12328,0.367020,-0.084040,0.038793,-0.006287,0.101690,-0.376340,0.142640,-0.088119,-0.056629,-0.479970,0.25017,0.20565,0.100910,0.090710,-0.041637,-0.33688,0.223350,0.19753,0.40784,0.221110,-0.017778,-0.019428,0.62700,0.48108,0.450290,0.45691,0.19346,0.70298,0.685360,0.543640,0.67527,0.244000,0.512400,0.396410,...,0.47717,0.27306,0.71654,0.561570,-0.091094,0.11545,0.420580,0.631940,0.731340,-0.274750,0.59071,0.83211,-0.032933,-0.025029,-0.31699,-0.094311,0.294930,0.188810,0.01650,0.210740,0.456090,0.091578,-0.47742,0.013968,0.20574,0.48151,0.49290,0.21511,0.44880,0.369300,0.59834,-0.477880,0.26634,0.582940,-0.041272,0.630050,0.70314,0.36383,-0.207880,0.64138
2,139149,0.24585,0.216620,-0.12468,-0.35380,0.16150,-0.002032,-0.133020,-0.035222,0.259040,-0.045302,-0.027871,0.424240,-0.361020,-0.256640,0.013372,0.24679,0.06869,0.122810,-0.491070,-0.016317,-0.29430,0.189520,0.37337,-0.33428,0.532240,-0.059252,-0.376230,0.46884,0.22889,0.203270,0.22383,0.31407,0.51565,0.079927,0.114280,0.35020,0.262910,0.018740,0.051793,...,-0.36236,0.32105,0.37874,-0.258550,-0.312070,0.59020,0.449510,0.041927,0.729640,-0.592430,0.32563,0.75814,0.075444,-0.420990,-0.17144,-0.175620,0.041856,0.042524,-0.34176,0.377860,0.071924,0.539140,-0.55848,0.152460,0.45270,0.55959,0.46717,-0.15872,0.17388,0.521330,0.22866,-0.000816,0.25860,0.280450,-0.644640,0.332440,0.87074,-0.10973,-0.679160,0.32316
3,146791,0.42090,0.331380,0.24453,0.17167,0.59223,0.431050,0.280290,0.289620,0.356800,0.286340,0.048550,-0.080789,-0.060777,0.260040,0.154730,0.16130,0.21598,0.153080,0.503730,0.148900,0.18326,0.021985,0.40514,0.52879,0.300160,0.323960,0.328850,0.74485,0.70518,0.658060,0.54178,0.38055,0.64302,0.298820,0.589520,0.53017,0.485810,0.574150,0.509770,...,-0.24084,0.50273,0.76808,-0.075026,0.190440,0.45596,0.161080,0.449680,0.609100,0.177100,0.42164,0.57511,0.521790,0.735700,0.27329,0.207670,0.425530,0.244690,0.23583,0.493320,0.599920,0.312640,0.26743,0.279880,0.42898,0.39231,0.13384,0.49211,0.34309,0.144460,0.42881,0.711570,0.56593,0.248560,0.153860,0.093171,0.62556,0.60391,-0.058301,0.10779
4,153870,-0.14621,-0.468630,-0.52880,-0.50381,-0.51052,-0.029113,-0.015192,0.360170,0.005944,0.024801,-0.031034,0.559150,0.048705,0.156080,0.008678,0.62856,0.55702,0.191120,0.040825,0.373060,-0.44247,0.119220,-0.34615,-0.51165,0.452220,-0.160560,-0.606830,0.28326,0.21766,0.158000,0.24057,0.28307,0.25707,-0.072729,0.152580,0.13349,-0.105510,-0.234190,0.183910,...,-0.39077,0.51207,0.29410,-0.453800,0.073685,0.18873,0.128680,-0.276260,0.202700,-0.045402,0.15311,0.20027,-0.353570,0.396270,-0.49929,-0.618850,0.569730,-0.323330,-0.57656,-0.286060,0.378360,0.753900,-0.84422,0.333820,0.68311,-0.23637,-0.35586,0.44029,-0.40981,-0.179310,0.41688,-0.435600

In [31]:
train_SBM.head()

,Id,SBM_map1,SBM_map2,SBM_map3,SBM_map4,SBM_map5,SBM_map6,SBM_map7,SBM_map8,SBM_map10,SBM_map13,SBM_map17,SBM_map22,SBM_map26,SBM_map28,SBM_map32,SBM_map36,SBM_map40,SBM_map43,SBM_map45,SBM_map48,SBM_map51,SBM_map52,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
0,120873,0.725065,-0.639254,0.353069,-0.981707,-1.419971,-0.441321,-0.264192,0.711579,0.641798,0.339658,0.137624,0.098926,-0.179555,-1.257417,-0.426855,-1.062109,0.866065,-0.409747,-0.388249,1.641117,0.439271,0.791762,-0.078220,-0.982331,1.070363,0.220316,0.776855,-2.022404,1.203256,1.083516,0.564201,-0.002006
1,135376,-1.328855,0.502970,0.013232,1.128496,-0.070738,0.398476,-0.466051,-0.435455,0.342847,-0.672254,0.972934,-0.177950,0.742952,1.175508,0.083895,0.044317,1.154924,1.034574,-0.612464,0.516993,-0.875616,-0.073326,0.989571,-0.057543,0.371701,-0.513081,-0.457550,1.476216,0.381052,0.163436,0.403782,-0.295125
2,139149,0.732268,-1.241554,0.654942,-0.289216,0.158316,0.029165,1.439242,-0.832816,1.285468,-2.132901,-1.488153,-0.202826,2.034083,0.323568,-1.676738,0.414747,0.498362,-1.212714,1.249961,0.507938,1.360176,-0.910225,-0.257114,0.597229,1.220756,-0.059213,-0.435494,-0.092971,1.090910,-0.448562,-0.508497,0.350434
3,146791,-0.343917,-1.052519,-1.150521,0.765989,0.923129,0.674052,-0.492673,1.282388,-1.914583,-1.269851,0.187573,-0.355915,0.064853,0.159363,-2.018822,-0.026555,-0.732960,-0.981925,0.440149,-3.060313,-0.331995,-3.013096,-0.295110,0.829697,-0.450726,-0.791032,-1.115821,-0.022296,-0.245844,-0.705539,0.061344,0.448966
4,153870,-0.208685,-0.562697,-0.362164,1.025571,0.151690,-0.130410,-1.105922,0.502029,-0.097735,1.561833,1.961955,-0.420548,0.956431,-0.873050,0.673888,-1.027496,-0.012389,0.082437,1.293312,0.577215,-0.499647,0.474353,1.342273,-0.978412,0.158492,0.889753,0.795368,0.738788,0.475415,2.340384,2.516038,-0.551440


In [33]:
y.head()

,Id,Class
0,120873,1
1,135376,0
2,139149,0
3,146791,0
4,153870,1


In [0]:
test_FNC = pd.read_csv('./data/test_FNC.csv')
test_SBM = pd.read_csv('./data/test_SBM.csv')
comp_ind_fMRI = pd.read_csv('./data/comp_ind_fMRI.csv')
comp_ind_sMRI = pd.read_csv('./data/comp_ind_sMRI.csv')
rs_fMRI_FNC_mapping = pd.read_csv('./data/rs_fMRI_FNC_mapping.csv')

In [35]:
test_FNC.head()

,Id,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,FNC10,FNC11,FNC12,FNC13,FNC14,FNC15,FNC16,FNC17,FNC18,FNC19,FNC20,FNC21,FNC22,FNC23,FNC24,FNC25,FNC26,FNC27,FNC28,FNC29,FNC30,FNC31,FNC32,FNC33,FNC34,FNC35,FNC36,FNC37,FNC38,FNC39,...,FNC339,FNC340,FNC341,FNC342,FNC343,FNC344,FNC345,FNC346,FNC347,FNC348,FNC349,FNC350,FNC351,FNC352,FNC353,FNC354,FNC355,FNC356,FNC357,FNC358,FNC359,FNC360,FNC361,FNC362,FNC363,FNC364,FNC365,FNC366,FNC367,FNC368,FNC369,FNC370,FNC371,FNC372,FNC373,FNC374,FNC375,FNC376,FNC377,FNC378
0,100004,0.476127,0.064466,0.053238,-0.608133,0.073988,-0.637039,0.113556,-0.192434,-0.004025,-0.060474,-0.256244,-0.239706,0.113166,-0.136103,-0.322507,0.299439,0.492948,0.233049,-0.312542,0.210541,-0.521582,0.323302,-0.161545,-0.122164,0.369105,0.640831,0.041661,0.196605,0.128142,0.304551,0.352879,0.416197,0.678723,0.277782,0.676364,-0.133499,0.102174,0.022148,0.136891,...,-0.323568,0.415299,0.580612,-0.296234,0.700717,-0.098439,0.185673,0.380953,0.433161,-0.405556,0.374340,0.495940,-0.059396,0.136818,-0.483997,0.555808,-0.356607,0.100753,-0.320402,-0.112517,0.460471,0.386452,-0.073664,0.001062,0.153605,0.734810,0.317646,-0.097210,-0.122540,0.150698,0.482131,0.022525,-0.034251,-0.103903,-0.497701,0.079286,0.265937,0.379066,-0.630483,-0.309032
1,100015,0.013833,0.267183,0.232178,-0.167151,-0.261327,0.191869,0.406493,0.088761,0.177048,0.036718,-0.656346,0.415506,-0.054457,-0.401238,-0.321115,0.186433,0.585713,0.437911,-0.307749,-0.029330,-0.134603,-0.289691,0.009930,0.603237,-0.005077,0.381680,-0.135795,0.187608,0.716543,0.315034,-0.408325,0.617800,0.770686,0.181813,0.787717,0.605197,0.366940,-0.299074,0.421012,...,-0.377365,0.286297,0.682911,0.095391,0.388988,0.311717,-0.231560,0.310637,0.782427,0.009134,0.152884,0.510705,0.463462,0.043892,-0.252089,0.079247,-0.426585,0.385604,0.623455,0.286064,0.263112,0.218903,-0.589047,-0.113545,0.311255,0.030508,0.380939,0.297206,0.537560,0.318086,0.387964,0.288442,0.503947,-0.221215,-0.949426,0.340575,0.767685,0.417164,0.194081,-0.218685
2,100026,-0.435452,0.046780,0.243742,0.397030,-0.147821,0.173620,-0.461963,-0.610736,0.419753,0.400985,-0.220077,0.287517,0.002372,-0.332512,0.008532,0.366046,0.562904,0.238146,0.014265,-0.425968,-0.499350,0.067169,0.292784,0.211195,0.563133,0.231455,-0.313893,0.561810,-0.012589,-0.108333,0.608448,0.309184,-0.058818,0.137444,0.316538,0.058696,0.081580,0.113453,-0.623393,...,-0.701107,0.416079,0.510617,0.125767,-0.218539,0.609401,0.073030,0.217984,0.485754,0.040364,0.600444,-0.003409,0.268395,-0.213212,0.294130,-0.090743,0.051929,0.244618,0.234822,0.167235,0.324517,0.168946,0.434138,0.008318,-0.315736,0.485861,0.731697,-0.249642,-0.361329,0.093333,-0.388256,0.052361,0.759901,0.478927,-0.164136,-0.096020,0.607880,0.099879,-0.429554,0.453645
3,100030,-0.204510,-0.036735,-0.760705,-0.740495,0.064668,0.349926,-0.273826,-0.174384,-0.120248,0.175618,-0.122505,0.379375,0.040945,0.106041,-0.150561,0.114257,0.553867,-0.371839,-0.495213,-0.274343,-0.496839,0.405451,0.293438,0.546172,-0.355862,-0.053801,-0.492059,0.082710,0.113686,0.675230,-0.263651,0.451183,0.537128,-0.226750,0.124338,0.304847,0.073727,0.278834,0.279822,...,-0.498579,-0.230756,0.376818,-0.510979,0.136534,-0.190452,0.526877,-0.316290,0.570783,-0.119866,-0.107237,0.632972,-0.376752,0.727788,-0.270263,-0.712554,0.202652,-0.372602,0.049370,-0.717746,0.392160,0.719406,-0.526902,0.461217,0.267554,0.314762,0.682848,0.291971,0.462893,0.442313,0.406804,0.169224,0.001855,0.324120,-0.212088,0.344033,0.234451,0.214809,-0.138947,0.436006
4,100047,0.599435,-0.166441,0.122431,0.011539,0.346906,-0.017430,-0.274734,0.211510,0.151012,-0.033434,-0.286008,-0.024012,-0.245284,-0.280592,-0.278775,0.555220,-0.241175,-0.184943,0.558418,-0.313958,0.044287,0.554710,0.249658,0.594015,-0.245845,-0.426244,0.252298,0.023092,0.672685,0.624100,0.080206,0.622279,0.294196,0.088422,0.332166,0.467401,0.447499,0.264495,0.373256,...,-0.490835,0.626051,-0.352886,-0.072454,0.234495,0.436001,0.459640,-0.428985,0.357173,0.026662,0.50874

In [36]:
test_SBM.head()

,Id,SBM_map1,SBM_map2,SBM_map3,SBM_map4,SBM_map5,SBM_map6,SBM_map7,SBM_map8,SBM_map10,SBM_map13,SBM_map17,SBM_map22,SBM_map26,SBM_map28,SBM_map32,SBM_map36,SBM_map40,SBM_map43,SBM_map45,SBM_map48,SBM_map51,SBM_map52,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
0,100004,1.154881,3.627246,0.146076,-0.677521,0.951169,0.482531,-2.404130,-0.715182,-0.475299,-0.896095,2.256762,0.567271,0.796897,-0.617875,0.616627,-2.011786,-0.514364,-0.000241,0.025560,-0.184531,-0.538563,0.139369,-0.451994,1.123770,2.083006,1.145440,-0.067608,1.202529,0.851587,0.451583,-0.159739,0.192076
1,100015,-1.608977,-0.125656,2.232224,1.013575,1.227375,0.296077,-0.612468,-1.610222,-0.136698,0.039142,1.711094,0.908302,-0.930560,-0.900517,-0.865214,0.185261,0.726653,1.492725,-1.508538,1.732360,4.317689,-2.084801,0.696987,1.397832,1.046136,-0.191733,-2.192023,-0.369276,0.822225,-0.109342,-0.580476,0.174160
2,100026,1.611464,1.859449,-0.959714,-0.713587,0.826513,-0.405989,-0.752907,-0.484180,-1.212576,0.679237,1.386814,0.974427,-0.362832,0.106783,-0.136973,-0.123830,0.589357,0.867600,2.357174,-1.374768,-1.572011,0.046525,0.160145,1.906989,-2.661633,-0.193911,0.440873,0.641739,0.918397,-0.758046,0.154702,-0.476647
3,100030,-0.283111,-0.418010,1.962834,-0.342130,-0.939108,0.929705,1.041755,-1.073416,-0.815344,-1.945322,-0.949757,1.275905,-1.389023,0.561610,2.226175,0.167515,0.375615,-0.318241,-2.162269,-0.135170,0.553775,-1.693663,0.974828,-1.997087,-2.083782,1.154107,-0.643947,2.332424,0.659124,-0.809445,0.558960,2.790871
4,100047,-0.688717,1.951002,1.512576,-0.854993,1.059652,-0.179238,-1.775324,0.649346,1.616116,0.985532,0.415556,-0.063359,1.356835,1.181061,-1.149081,2.410666,-0.125726,-2.925785,1.400921,-0.319947,1.480964,0.021838,-0.789153,1.578984,1.402592,-1.230440,0.296686,2.806314,0.427184,-0.240682,-0.196948,-1.544345


In [37]:
comp_ind_fMRI.head()

,fMRI_comp_ind
0,7
1,17
2,20
3,21
4,23


In [38]:
comp_ind_sMRI.head()

,sMRI_comp_ind
0,1
1,2
2,3
3,4
4,5


In [39]:
rs_fMRI_FNC_mapping.head()

,FNC,mapA,mapB
0,1,21,17
1,2,21,7
2,3,21,23
3,4,21,24
4,5,21,38


In [0]:
train = pd.merge(train_FNC, train_SBM, how='left', on='Id')
test = pd.merge(test_FNC, test_SBM, how='left', on='Id')

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Columns: 411 entries, Id to SBM_map75
dtypes: float64(410), int64(1)
memory usage: 276.8 KB


In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119748 entries, 0 to 119747
Columns: 411 entries, Id to SBM_map75
dtypes: float64(410), int64(1)
memory usage: 376.4 MB


In [43]:
print('Classes:\n', y['Class'].unique())

Classes:
 [1 0]


In [0]:
sns.pairplot(train_FNC, hue = 'continent')